In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Final_Project/data/recipe.csv', encoding='cp949')
df.head()

,메뉴,재료,양,레시피
0,고추전,"['아삭이고추', '다진 돼지고기', '두부', '양파', '부침가루', '계란 흰...","['7~8개', '150g', '1/2모(150g)', '1/4개', '1큰술', ...",['양파는 곱게 다지고 홍고추는 송송 썰어 준비해 주세요. 두부는 칼등으로 으깨 주...
1,황태갈비,"['황태', '밀가루', '전분', '식용유', '간장', '올리고당', '갈색 설...","['1마리', '4큰술', '2큰술', '약간', '2큰술', '1큰술', '1큰술...",['뼈를 발라 펼쳐진 황태를 6cm 길이로 잘라주세요. 물에 담가 말랑한 상태로 만...
2,성게미역국,"['성게', '건 미역', '다시마 육수', '국간장', '참기름', '멸치액젓',...","['60g', '15g', '5컵', '1큰술', '1큰술', '1/2큰술', '약간']",['미역은 물에 담가 10분 정도 불려주세요. 불린 미역은 먹기 좋게 썬 후 물기를...
3,멍게비빔밥,"['멍게', '부추', '청양고추', '밥', '다진 마늘', '소금', '참기름'...","['120g', '한 줌(30g)', '1개', '2공기', '1큰술', '1/2작...","['손질된 멍게는 먹기 좋은 크기로 잘라주세요.', '쪽파와 청양고추는 송송 썰어주..."
4,곤드레전,"['생 곤드레', '식용유', '소금', '부침가루', '물', '간장', '맛술'...","['200g', '적당량', '약간', '1컵', '1컵', '1큰술', '1/2큰...","['곤드레는 깨끗이 씻어 4cm 길이로 잘라주세요.', '볼에 반죽 재료와 곤드레를..."


In [ ]:
import ast

cook_list = [] # 레시피 목록
ingredients = [] # 레시피별 재료 목록
all_ingredients = {} # 전체 재료 목록
for idx, row in df.iterrows(): ## pandas dataframe의 한 행씩 가져옴. idx=행 번호(0부터), row=행의 data
    cook_list.append(row['메뉴'])
    ingredient_map = {}
    for val in ast.literal_eval(row['재료']):
        ingredient_map[val] = 1
        all_ingredients[val] = 1
    ingredients.append(ingredient_map)

In [ ]:
category_index = {} ## 재료의 카테고리 번호를 저장 (중복X)
for idx, val in enumerate(sorted(all_ingredients.keys())):
    category_index[val] = idx

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

matrix = []
for ingred_map in ingredients:
    tmp = []
    for val in ingred_map.keys(): ## 재료 번호를 전부 저장 -> [10, 21, 34, ...]
        tmp.append(category_index[val])
    matrix.append(tmp)

<ipython-input-4-f4ead27a87ef>:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  matrix = np.array(matrix) ## 넘파이 어레이로 바꿔줌


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

## [[10, 21, 34, ...], [30, 40, ...]] => [[1, 0, 0, 1], [1, 1, 0, 0],...] 과 같은 형태로 0, 1로 구성된 행렬로 변환함
mlb = MultiLabelBinarizer()
multi_hot_matrix = mlb.fit_transform(matrix)

print(multi_hot_matrix)

[[0 0 0 ... 0 0 1]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
# 사용자 레시피 벡터 (예시)
user_recipe = multi_hot_matrix[1]

# 모든 레시피와 유사도 계산
similarity_scores = cosine_similarity([user_recipe], multi_hot_matrix)[0]

In [ ]:
user_recipe

array([0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0])

In [ ]:
similarity_scores

array([0.12598816, 1.        , 0.10910895, 0.20412415, 0.34815531,
       0.        , 0.18257419, 0.28867513, 0.10910895, 0.10206207,
       0.21650635, 0.        , 0.25      , 0.1490712 , 0.39735971,
       0.3086067 , 0.33333333, 0.1490712 , 0.10910895, 0.28867513,
       0.07715167, 0.54006172, 0.46291005, 0.09128709, 0.36514837,
       0.23145502, 0.13608276, 0.14433757, 0.17407766, 0.0745356 ,
       0.25      , 0.2236068 , 0.09128709, 0.19245009, 0.09128709,
       0.20412415, 0.15430335, 0.09128709, 0.09128709, 0.10910895,
       0.12909944, 0.        , 0.        , 0.        , 0.42008403,
       0.        , 0.24019223, 0.08703883, 0.10206207, 0.2236068 ])

In [ ]:
 ## 유사도를 오름 차순으로 정렬한 인덱스를 내림 차순으로 변환하여 저장
idxs = np.argsort(similarity_scores)[::-1]

## 상위 5개 점수
top5_scores = similarity_scores[idxs][:5]
top5_idxs = idxs[:5] ## 상위 5개 인덱스 번호
top5_recipe = [] ## 상위 5개 매칭 레시피

for id in top5_idxs:
    top5_recipe.append(cook_list[id])

In [ ]:
top5_recipe

['황태갈비', '표고버섯 덮밥', '냉이 차돌박이 솥밥', '꼬막장', '채식구절판']